In [1]:
import QDMPy.data_loading as data_loading
import QDMPy.fit_interface as fit_interface
import QDMPy.fit_plots as fit_plots

import matplotlib
import matplotlib.pyplot as plt

# '%matplotlib widget' allows interactive plots in jupyterlab
# use ''%matplotlib inline' to allow export to pdf
%matplotlib widget

# Load the data in

In [2]:
options_dict = {
    "base_dir": "/home/samsc/ResearchData/test_images/",
    "filepath": "ODMR - Pulsed with IR_83",
    
    "additional_bins": 8,

    "system_name": "Zyla",

    "ROI": "Rectangle", 
    "ROI_start": [0, 0],
    "ROI_end": [500,500],

    "normalisation": "div", # fix this for consistent fitting -> get to 0->1?
    
    "single_pixel_check": [150,150],
    "AOI_1_start": [100, 100],
    "AOI_1_end": [103, 103], 
    "AOI_2_start": [200, 200],
    "AOI_2_end": [203, 203], 
    "AOI_3_start": [150, 100],
    "AOI_3_end": [153, 103], 

    "fit_backend": "scipy",
    "fit_backend_comparison": ["scipy"],
    "fit_pixels": True, 
    "force_fit": True, 
    "auto_match_prev_ROI_options": False,
    
    "use_ROI_avg_fit_res_for_all_pixels": True,
    
    "scipy_sub_threads": 2, 

    "fit_functions":  {"linear": 1, "lorentzian": 8},

    "pos_guess": [2610, 2708, 2805, 2879, 2953, 3020, 3092, 3145],
    "pos_range": 25,

    "amp_guess": -0.0015,
    "amp_bounds": [-0.0100, -0.0003],

    "fwhm_guess": 9,
    "fwhm_bounds": [5, 20],

    "c_guess": 1,
    "c_bounds": [0.9995, 1.0005],

    "m_guess": 0,
    "m_bounds": [-1e-6, 1e-6],
    
    "scramble_pixels": True,
    
    "gpufit_tolerance": 1e-12,
    "gpufit_max_iterations": 25,
    "gpufit_estimator_id": "LSE"
}

In [3]:
# QDMPy_path = data_loading.DIR_PATH
# options = data_loading.load_options(path=QDMPy_path / "options/test_options_odmr.json", check_for_prev_result=True)
options = data_loading.load_options(options_dict=options_dict, check_for_prev_result=True)
fit_plots.set_mpl_rcparams(options)

raw_data, prelim_sweep_list = data_loading.load_raw_and_sweep(options)

PL_image, PL_image_ROI, sig, ref, sig_norm, single_pixel_pl, sweep_list = data_loading.reshape_dataset(
        options, raw_data, prelim_sweep_list
    )

/home/samsc/src/nv/QDMPy_proj/QDMPy_git/QDMPy/data_loading.py:327: UserWarning: ROI Rectangle too big in x, cropping to image.
  warnings.warn(f"{name} Rectangle too big in x, cropping to image.")
/home/samsc/src/nv/QDMPy_proj/QDMPy_git/QDMPy/data_loading.py:330: UserWarning: ROI Rectangle too big in y, cropping to image.
  warnings.warn(f"{name} Rectangle too big in y, cropping to image.")
/home/samsc/src/nv/QDMPy_proj/QDMPy_git/QDMPy/data_loading.py:313: UserWarning: AOI_1 Rectangle starts outside image (in x), setting to zero.
  warnings.warn(f"{name} Rectangle starts outside image (in x), setting to zero.")
/home/samsc/src/nv/QDMPy_proj/QDMPy_git/QDMPy/data_loading.py:320: UserWarning: AOI_1 Rectangle starts outside image (in y), setting to zero.
  warnings.warn(f"{name} Rectangle starts outside image (in y), setting to zero.")
/home/samsc/src/nv/QDMPy_proj/QDMPy_git/QDMPy/data_loading.py:327: UserWarning: AOI_1 Rectangle too big in x, cropping to image.
  warnings.warn(f"{name} Re

In [4]:
fig1 = fit_plots.plot_ROI_PL_image(options, PL_image)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
AOIs = data_loading.define_AOIs(options)
fig2 = fit_plots.plot_AOI_PL_images(options, PL_image_ROI)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
aoi_spectra_plot = fit_plots.plot_AOI_spectra(options, AOIs, sig, ref, sweep_list)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Fit AOI, ROI averages, single pixel

In [7]:
fit_model = fit_interface.define_fit_model(options)
backend_ROI_results_lst = fit_interface.fit_ROI_avg(options, sig_norm, sweep_list, fit_model)

fig3 = fit_plots.plot_ROI_avg_fits(options, backend_ROI_results_lst)

/home/samsc/src/nv/QDMPy_proj/QDMPy_git/QDMPy/fit_scipy.py:26: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm  # auto detects jupyter


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
fit_result_collection_lst = fit_interface.fit_AOIs(options, sig_norm, single_pixel_pl, sweep_list, fit_model, AOIs, backend_ROI_results_lst)

AOI_spectra_fit_fig = fit_plots.plot_AOI_spectra_fit(options, sig, ref, sweep_list, AOIs, fit_result_collection_lst, fit_model)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Fit pixels

In [9]:
if (options["force_fit"] or not options["found_prev_result"]) and options["fit_pixels"]:
    pixel_fit_params = fit_interface.fit_pixels(options, sig_norm, sweep_list, fit_model, backend_ROI_results_lst[0])
elif options["found_prev_result"]:
    pixel_fit_params = fit_interface.load_prev_fit_results(options)
else:
    pixel_fit_params = None  # not fitting pixels, this stops plotting (e.g. via plot_param_images) from erroring

  0%|          | 0/4096 [00:00<?, ? PX/s]

# Plot (fit) parameters

In [10]:
fit_plots.plot_param_images(options, fit_model, pixel_fit_params, "c")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
fit_plots.plot_param_images(options, fit_model, pixel_fit_params, "m")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
fit_plots.plot_param_images(options, fit_model, pixel_fit_params, "pos")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
fit_plots.plot_param_images(options, fit_model, pixel_fit_params, "amp")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
fit_plots.plot_param_images(options, fit_model, pixel_fit_params, "fwhm")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
data_loading.save_options(options)
options

{'base_dir': '/home/samsc/ResearchData/test_images/',
 'filepath': '/home/samsc/ResearchData/test_images/ODMR - Pulsed with IR_83',
 'raw_pixel_size': 2.44140625e-07,
 'ignore_ref': False,
 'additional_bins': 8,
 'old_binning_convention': False,
 'system_name': 'Zyla',
 'ROI': 'Rectangle',
 'ROI_start': (0, 0),
 'ROI_end': (63, 63),
 'remove_start_sweep': 1,
 'remove_end_sweep': 0,
 'normalisation': 'div',
 'single_pixel_check': (32, 32),
 'AOI_1_start': (0, 0),
 'AOI_1_end': (63, 63),
 'AOI_2_start': (0, 0),
 'AOI_2_end': (63, 63),
 'AOI_3_start': (0, 0),
 'AOI_3_end': (63, 63),
 'AOI_4_start': None,
 'AOI_4_end': None,
 'AOI_5_start': None,
 'AOI_5_end': None,
 'fit_backend': 'scipy',
 'fit_backend_comparison': ['scipy'],
 'force_fit': True,
 'fit_pixels': True,
 'auto_match_prev_ROI_options': False,
 'scramble_pixels': True,
 'use_ROI_avg_fit_res_for_all_pixels': True,
 'fit_functions': {'linear': 1, 'lorentzian': 8},
 'pos_guess': [2610, 2708, 2805, 2879, 2953, 3020, 3092, 3145],
 